In [1]:
from solver import *

### Compute the matrix $Z_\pi(N)$

In [2]:
a, b, x, y, t = sp.symbols('a b x y t')

In [3]:
# Create solvers and set dictionary
s_KVar_k = Solver()
s_KVar_t = Solver()

# S = [ (a, t) : a^2 = t ]
S = sp.Symbol('S')

s_KVar_t.dictionary.append(([ sp.Poly(t, t) ], 0))
s_KVar_t.dictionary.append(([ sp.Poly(t - 1, t) ], 0))
s_KVar_t.dictionary.append(([ sp.Poly(a**2 - t, t, a) ], S))
s_KVar_t.dictionary.append(([ sp.Poly(a**2*b**2 - t, t, a, b) ], (q - 1) * S))

In [4]:
# Compute Z_pi_N
Z_pi_N = sp.Matrix([[ 0, 0, 0 ], [ 0, 0, 0 ], [ 0, 0, 0 ]])

Z_pi_N[0,0] = s_KVar_k.compute_class(System([ a**2 - 1, a*b + b ], [ a ], { a, b }))
Z_pi_N[1,0] = s_KVar_k.compute_class(System([ a**2 - 1 ], [ a, a*b + b ], { a, b }))
Z_pi_N[2,0] = s_KVar_t.compute_class(System([ a**2 - t ], [ a ], { a, b, t }, { t }))

Z_pi_N[0,1] = s_KVar_k.compute_class(System([ a**2 - 1, a*b + b + x ], [ x, a ], { a, b, x }))
Z_pi_N[1,1] = s_KVar_k.compute_class(System([ a**2 - 1 ], [ x, a,  a*b + b + x ], { a, b, x }))
Z_pi_N[2,1] = s_KVar_t.compute_class(System([ a**2 - t ], [ x, a ], { a, b, x, t }, { t }))

Z_pi_N[0,2] = s_KVar_k.compute_class(System([ a**2*y**2 - 1, y**2*(a*b + b) + x ], [ y, y**2 - 1, a ], { a, b, y, x }))
Z_pi_N[1,2] = s_KVar_k.compute_class(System([ a**2*y**2 - 1 ], [ y, y**2 - 1, a, y**2*(a*b + b) + x ], { a, b, y, x }))
Z_pi_N[2,2] = s_KVar_t.compute_class(System([ a**2*y**2 - t ], [ y, y**2 - 1, a ], { a, b, y, x, t }, { t }))

In [5]:
# Factor entries + for the relevant matrix, just evaluate S to 1
for i in range(3):
    for j in range(3):
        Z_pi_N[i, j] = sp.factor(Z_pi_N[i, j].subs(S, 1))
        
Z_pi_N

Matrix([
[q + 1,             q - 1,         2*q*(q - 3)],
[q - 1, (q - 1)*(2*q - 1), 2*q*(q - 3)*(q - 1)],
[    q,         q*(q - 1),        q**2*(q - 3)]])

### Determine $\tilde{Z}(N) = Z_\pi(N) \circ \eta^{-1}$ and diagonalize

In [6]:
# Construct eta and eta inverse
eta = sp.Matrix([[ 1, 0, 0 ], [ 0, q - 1, 0], [ 0, 0, q ]])
eta_inv = sp.Matrix([[ 1, 0, 0 ], [ 0, 1 / (q - 1), 0], [ 0, 0, 1 / q ]])

# Compute Z_tilde_N = Z_pi_N * eta_inv
Z_tilde_N = sp.expand(sp.simplify(Z_pi_N * eta_inv))

Z_tilde_N

Matrix([
[q + 1,       1,          2*q - 6],
[q - 1, 2*q - 1, 2*q**2 - 8*q + 6],
[    q,       q,       q**2 - 3*q]])

In [7]:
# Diagonalize Z_tilde_N
P, D = Z_tilde_N.diagonalize()

In [8]:
P

Matrix([
[   (3 - q)/q, -1,     2/q],
[-q + 4 - 3/q,  1, 2 - 2/q],
[           1,  0,       1]])

In [9]:
D

Matrix([
[0, 0,         0],
[0, q,         0],
[0, 0, q*(q - 1)]])

In [10]:
# Obtain general formula by taking powers
r = sp.Symbol('r')
formula = (P * sp.Matrix([[ 0, 0, 0 ], [ 0, q**r, 0 ], [ 0, 0, q**r * (q - 1)**r ]]) * P.inv())[0, 0]

formula

q**r*(q - 1)/q + 2*q**r*(q - 1)**r/(q*(q - 1))

### Manually compute class for small cases, and compare to the obtained formula

In [11]:
a, b, c, d, e, f, g, h, i, j = sp.symbols('a b c d e f g h i j')

A = sp.Matrix([[ a, b ], [ 0, 1 ]])
B = sp.Matrix([[ c, d ], [ 0, 1 ]])
C = sp.Matrix([[ e, f ], [ 0, 1 ]])
D = sp.Matrix([[ g, h ], [ 0, 1 ]])
E = sp.Matrix([[ i, j ], [ 0, 1 ]])

In [12]:
solver = Solver()

c_XN = [ 0 ] * 6

c_XN[0] = 1
M = A**2
c_XN[1] = sp.factor(solver.compute_class(System([ M[0,0] - 1, M[0,1] ], [ a ], { a, b })))
M = A**2 * B**2
c_XN[2] = sp.factor(solver.compute_class(System([ M[0,0] - 1, M[0,1] ], [ a, c ], { a, b, c, d })))
M = A**2 * B**2 * C**2
c_XN[3] = sp.factor(solver.compute_class(System([ M[0,0] - 1, M[0,1] ], [ a, c, e ], { a, b, c, d, e, f })))
M = A**2 * B**2 * C**2 * D**2
c_XN[4] = sp.factor(solver.compute_class(System([ M[0,0] - 1, M[0,1] ], [ a, c, e, g ], { a, b, c, d, e, f, g, h })))
M = A**2 * B**2 * C**2 * D**2 * E**2
c_XN[5] = sp.factor(solver.compute_class(System([ M[0,0] - 1, M[0,1] ], [ a, c, e, g, i ], { a, b, c, d, e, f, g, h, i, j })))

for i in range(6):
    print('[ X(N_{}) ] = {}'.format(i, c_XN[i]))

# [ X(N_0) ] = 1
# [ X(N_1) ] = q + 1
# [ X(N_2) ] = 3*q*(q - 1)
# [ X(N_3) ] = q**2*(q - 1)*(2*q - 1)
# [ X(N_4) ] = q**3*(q - 1)*(2*q**2 - 4*q + 3)
# [ X(N_5) ] = q**4*(q - 1)*(2*q**3 - 6*q**2 + 6*q - 1)

[ X(N_0) ] = 1
[ X(N_1) ] = q + 1
[ X(N_2) ] = 3*q*(q - 1)
[ X(N_3) ] = q**2*(q - 1)*(2*q - 1)
[ X(N_4) ] = q**3*(q - 1)*(2*q**2 - 4*q + 3)
[ X(N_5) ] = q**4*(q - 1)*(2*q**3 - 6*q**2 + 6*q - 1)


In [13]:
formula = q**(r - 1) * (q - 1) + 2*q**(r - 1) * (q - 1)**(r - 1)

for i in range(1, 6):
    print(sp.expand(c_XN[i] - formula.subs(r, i)) == 0)

True
True
True
True
True
